Requirements:

```
py2neo==2020.0.0
```

In [ ]:
import os
import py2neo
py2neo.__version__

In [ ]:
graph = py2neo.Graph(
    "bolt://localhost:7687", 
    password=os.getenv("PASSWORD")
)

### Did the connection work?

In [ ]:
graph.run("MATCH (n) RETURN n").data()

### Running a query:

In [ ]:
query = """
  MATCH 
    (people:Person)
    -[work:WORKS_AT]-
    (dept:Department) 
    
  RETURN people.name, work, dept.name
"""
graph.run(query)

In [ ]:
data = graph.run(query)
data.to_table()

### Using built-in `match()` function:

In [ ]:
graph.nodes.match("Person").all()

In [ ]:
graph.nodes.match(name="Alice").first()

In [ ]:
graph.nodes.match("AllRoundLegend").first()

In [ ]:
employees = graph.match(r_type="WORKS_AT").all()
for employee in employees:
    print(employee)

In [ ]:
query = """
MATCH 
  (dept:Department {name: "IT Department"})
   <-[:WORKS_AT]-(people:Person)-[:LIKES]->
  (framework:Framework)
   -[:LANGUAGE]->
  (python:Language {name: "Python"})
  
RETURN framework.name, people.name
ORDER BY framework.name
"""
employees = graph.run(query).data()
for employee in employees:
    print(employee)


### Recommendations Statement

In [ ]:
statement_recommend = """
MATCH (subject:Person)
WHERE subject.name=$person_name      
      
MATCH
     (dept:Department {name: "IT Department"})
     -[:WORKS_AT]-
     (people:Person)
     -[:LIKES]-
     (likeable_framework:Framework)
     -[:LANGUAGE]-
     (:Language {name: "Python"})
     
WHERE NOT (subject)-[:LIKES]->(likeable_framework)

RETURN DISTINCT likeable_framework.name
"""
graph.run(statement_recommend, person_name="Cris")